In [2]:
from rubik_cubes import Rubik, rubik_corner, rubik_edge
from rubik_state import rubik_state
from rubik_next import get_nexts_2

In [3]:
def compress_state(state):
    compressed = 0
    # first compress corners place
    for i in range(1, 9):
        compressed <<= 3
        compressed |= state.corners[i].final_position
    # compress corners orientation
    for i in range(1, 9):
        compressed <<= 2
        compressed |= state.corners[i].orientation
    # compress edges place
    for i in range(1, 13):
        compressed <<= 4
        compressed |= state.edges[i].final_position
    # corners edges orientation
    for i in range(1, 13):
        compressed <<= 1
        compressed |= state.edges[i].orientation
    return str(compressed)

# forbidden_move = {
#     0: [0],
#     1: [1, 7, 13],
#     7: [1, 7, 13],
#     13: [1, 7, 13],
#     4: [4, 10, 16],
#     10: [4, 10, 16],
#     16: [4, 10, 16],
#     14: [14],
#     15: [15],
#     17: [17],
#     18: [18],
# }

# def get_nexts_2(state):
#     nexts = []
# #     for i in [13, 14, 15, 16, 17, 18, 1, 4, 7, 10]:
# #         next = rubik_state(state, i)
# #         nexts.append(next)
#     return [rubik_state(state, i) for i in [13, 14, 15, 16, 17, 18, 1, 4, 7, 10] if i not in forbidden_move[state.instruction]]


In [4]:
def gen_pruning(dico, curr_node, heuristic, depth):
    if depth < 5:
        next_depth = []
        for elem in get_nexts_2(curr_node):
            dico[compress_state(elem)] = heuristic(elem)
            next_depth.append(elem)
        for elem in next_depth:
            gen_pruning(dico, elem, heuristic, depth + 1)

In [6]:
dico = {}
rubik = Rubik()
state = rubik_state(rubik, 0)
# state2 = rubik_state(state, 1)
# for elem in get_nexts_2(state2):
#     print(elem.instruction)
dep = rubik_state(rubik, 0)
gen_pruning(dico, dep, rubik.heuristic_h2, 0)

In [7]:
len(dico)

25349

In [68]:
import pandas as pd

pruning_table = {}
pruning_table["pruning"] = dico
df = pd.DataFrame.from_dict(pruning_table)
df.to_hdf("phase2.h5", key="df", complevel=9, complib='blosc:lz4')